This is a trial notebook for approx pipeline and to test models, ignore the data and EDA

In [23]:
import numpy as np 
import pandas as pd

In [24]:
df_train = pd.read_csv("../species_prediction_model/data/trial_data_riya.csv")
df_train.head()

,tag_id_long,watershed,river,site,method,local,water_temp_start,species,fork_length_mm,weight_g
0,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,NaN,co,98,NaN
1,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,NaN,co,115,NaN
2,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,NaN,rbt,106,NaN
3,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,7.0,co,113,NaN
4,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,7.0,rbt,81,NaN


In [25]:
df_train.isnull().sum()

tag_id_long           0
watershed             0
river                 0
site                  0
method                0
local                 0
water_temp_start      3
species               0
fork_length_mm        0
weight_g            100
dtype: int64

In [26]:
df_train.species.unique()

array(['co', 'rbt'], dtype=object)

This dataset only has 2 labels - co and rbt

In [27]:
for col in df_train.columns:
    if col not in ['tag_id_long']:
        print(col, ":", df_train[col].unique())

watershed : ['englishman']
river : ['shelly creek']
site : ['martindale rd']
method : ['smolt trap']
local : ['in-river']
water_temp_start : [ nan  7.   7.8  8.6 10.5 12.  10.6]
species : ['co' 'rbt']
fork_length_mm : [ 98 115 106 113  81  76 100  86  87  91  95  99 101  84  88  89  90 107
 164  93  73  74  80  82  83  85  94  97 104  96 102 105  77 119  92  78]
weight_g : [nan]


In [28]:
df_train = df_train.drop('weight_g, tag_id_long', axis = 1)
df_train = df_train.dropna()

In [ ]:
df_train

,tag_id_long,watershed,river,site,method,local,water_temp_start,species,fork_length_mm
3,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,7.0,co,113
4,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,7.0,rbt,81
5,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,7.8,co,76
6,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,7.8,co,100
7,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,8.6,co,86
...,...,...,...,...,...,...,...,...,...
95,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,10.6,co,100
96,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,10.6,co,101
97,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,10.6,rbt,78
98,989.001039,englishman,shelly creek,martindale rd,smolt trap,in-river,10.6,rbt,104
